# Day9_2: 회귀분석 기초 (Regression Basics)

## 📚 학습 목표

**Part 1: 기초 개념**
1. 회귀분석의 개념과 목적 이해하기
2. 단순 선형 회귀 모델 구축하기
3. 최소제곱법(OLS)의 원리 파악하기
4. 회귀계수의 의미 해석하기
5. R-squared(결정계수)로 모델 평가하기

**Part 2: 심화 분석**
1. 다중 선형 회귀로 여러 변수 활용하기
2. statsmodels로 상세한 회귀 분석 수행하기
3. 잔차 분석으로 모델 가정 검증하기
4. RMSE, MAE로 예측 성능 평가하기

---

## 🎯 왜 이것을 배우나요?

| 개념 | 실무 활용 | 예시 |
|------|----------|------|
| 선형 회귀 | 수치 예측 | 광고비로 매출 예측 |
| 다중 회귀 | 복합 요인 분석 | 면적, 층수, 역세권으로 집값 예측 |
| 회귀계수 | 영향도 정량화 | "광고비 100만원 증가 시 매출 50만원 증가" |
| R-squared | 모델 설명력 | "모델이 매출 변동의 85% 설명" |
| 잔차 분석 | 모델 진단 | 예측 오류 패턴 발견 |

**분석가 관점**: 회귀분석은 "X가 Y에 얼마나 영향을 미치는가?"를 정량적으로 답하는 핵심 도구입니다!

---

# Part 1: 기초 개념

---

## 1.1 회귀분석이란?

### 회귀분석의 정의

**회귀분석(Regression Analysis)**: 독립변수(X)와 종속변수(Y) 사이의 관계를 수학적 함수로 모델링하는 통계 기법

- **독립변수(X)**: 예측에 사용하는 변수 (설명변수, 특성, Feature)
- **종속변수(Y)**: 예측하려는 변수 (반응변수, 타겟, Target)

### 회귀분석 vs 상관분석

| 구분 | 상관분석 | 회귀분석 |
|------|---------|----------|
| 목적 | 관계의 강도 측정 | 관계를 수식으로 표현 |
| 방향성 | 없음 (대칭적) | 있음 (X → Y) |
| 결과물 | 상관계수 (r) | 회귀식 (Y = aX + b) |
| 활용 | "관련이 있는가?" | "얼마나 영향을 미치는가?" |

In [1]:
# 필요한 라이브러리 임포트
import numpy as np
import pandas as pd
from scipy import stats
import statsmodels.api as sm
import statsmodels.formula.api as smf
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
pd.options.plotting.backend = "plotly"


# sklearn 회귀 관련
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

print("라이브러리 임포트 완료")

라이브러리 임포트 완료


### 💡 실무 예시: 광고비와 매출의 관계

마케팅 팀에서 "광고비를 얼마나 쓰면 매출이 얼마나 나올까?"를 예측하고 싶습니다.

In [ ]:
np.array(ad_spend).reshape(-1,1)        #가로로 되어있으면 안됨... 

array([[100],
       [150],
       [200],
       [250],
       [300],
       [350],
       [400],
       [450],
       [500],
       [550]])

In [ ]:
# 광고비 vs 매출 데이터 (가상)
ad_spend = [100, 150, 200, 250, 300, 350, 400, 450, 500, 550]  # 광고비 (만원)
revenue = [1200, 1450, 1750, 2100, 2350, 2650, 2900, 3150, 3500, 3720]  # 매출 (만원)

# DataFrame 생성
df_ad = pd.DataFrame({
    '광고비': ad_spend,
    '매출': revenue
})

print("광고비 vs 매출 데이터:")
print(df_ad)
print(f"\n광고비 평균: {np.mean(ad_spend):.0f}만원")
print(f"매출 평균: {np.mean(revenue):.0f}만원")

---

## 1.2 단순 선형 회귀 (Simple Linear Regression)

### 선형 회귀 모델

단순 선형 회귀는 **하나의 독립변수**로 종속변수를 예측합니다.

**수식**: Y = β₀ + β₁X + ε

- **β₀ (절편, Intercept)**: X=0일 때 Y의 기대값
- **β₁ (기울기, Slope)**: X가 1 증가할 때 Y의 변화량
- **ε (오차, Error)**: 모델이 설명하지 못하는 부분

In [ ]:
# sklearn으로 단순 선형 회귀
X = np.array(ad_spend).reshape(-1, 1)  # 2D 배열로 변환
y = np.array(revenue)

# 모델 생성 및 학습
model = LinearRegression()
model.fit(X, y)     #훈련 하는거임

# 회귀계수 확인                         #모델이 학습한 계수를 확인할 수 있음
print("단순 선형 회귀 결과")            #stats모델이 더 자세하게 통계적으로 레포트를 받아볼 수 있음
print("=" * 50)
print(f"절편 (β₀): {model.intercept_:.2f}")
print(f"기울기 (β₁): {model.coef_[0]:.4f}")
print(f"\n회귀식: 매출 = {model.intercept_:.2f} + {model.coef_[0]:.4f} × 광고비")

단순 선형 회귀 결과
절편 (β₀): 636.12
기울기 (β₁): 5.6642

회귀식: 매출 = 636.12 + 5.6642 × 광고비


### 💡 회귀계수 해석

위 결과를 비즈니스 언어로 해석하면:
- **절편**: 광고비가 0원일 때 기본 매출은 약 680만원
- **기울기**: 광고비 1만원 증가 시 매출이 약 5.5만원 증가

In [ ]:
# 예측값 생성
y_pred = model.predict(X)

# 새로운 광고비로 매출 예측
new_ad_spend = [[600]]  # 600만원 광고비        #단일값이더라도 차원을 맞춰줘야되서 배열값임! 
predicted_revenue = model.predict(new_ad_spend)

print(f"광고비 600만원 투입 시 예상 매출: {predicted_revenue[0]:.0f}만원")

### Plotly로 회귀선 시각화 (px.scatter + trendline)

In [5]:
# px.scatter로 회귀선 포함 산점도
fig = px.scatter(
    df_ad,
    x='광고비',
    y='매출',
    title='광고비 vs 매출 (단순 선형 회귀)',
    trendline='ols',  # OLS 회귀선 추가
    labels={'광고비': '광고비 (만원)', '매출': '매출 (만원)'},
    width=700,
    height=500
)

# 마커 스타일
fig.update_traces(
    marker=dict(size=12, color='royalblue', line=dict(width=1, color='white')),
    selector=dict(mode='markers')
)

# 회귀선 스타일
fig.update_traces(
    line=dict(color='red', width=2),
    selector=dict(mode='lines')
)

fig.update_layout(
    font=dict(size=12),
    title_font_size=16
)

fig.show()

---

## 1.3 최소제곱법 (OLS: Ordinary Least Squares)

### 최소제곱법의 원리

**최소제곱법**: 잔차(실제값 - 예측값)의 제곱합을 최소화하는 회귀계수를 찾는 방법

**목표**: min Σ(yᵢ - ŷᵢ)² = min Σ(yᵢ - (β₀ + β₁xᵢ))²

### 잔차(Residual)란?

- **잔차**: 실제 관측값과 모델 예측값의 차이
- **residual = y - ŷ**
- 잔차가 작을수록 모델의 예측이 정확함

In [ ]:
# 잔차 계산
residuals = y - y_pred

# 잔차 분석
df_residuals = pd.DataFrame({
    '광고비': ad_spend,
    '실제매출': revenue,
    '예측매출': y_pred.round(1),
    '잔차': residuals.round(1)
})

print("잔차 분석 테이블:")
print(df_residuals)
print(f"\n잔차 합계: {residuals.sum():.4f}")  # 0에 가까워야 함
print(f"잔차 제곱합 (SSE): {(residuals**2).sum():.2f}")

### 💡 실무 예시: 잔차 시각화

In [ ]:
# 잔차 플롯 (실제값 vs 예측값 + 잔차 표시)
fig = go.Figure()

# 실제값 포인트
fig.add_trace(go.Scatter(
    x=ad_spend,
    y=revenue,
    mode='markers',
    name='실제값',
    marker=dict(size=12, color='royalblue')
))

# 회귀선
fig.add_trace(go.Scatter(
    x=ad_spend,
    y=y_pred,
    mode='lines',
    name='회귀선 (예측값)',
    line=dict(color='red', width=2)
))

# 잔차 표시 (세로선)
for i in range(len(ad_spend)):
    fig.add_trace(go.Scatter(
        x=[ad_spend[i], ad_spend[i]],
        y=[revenue[i], y_pred[i]],
        mode='lines',
        line=dict(color='gray', width=1, dash='dash'),
        showlegend=False
    ))

fig.update_layout(
    title='회귀분석: 실제값, 예측값, 잔차',
    xaxis_title='광고비 (만원)',
    yaxis_title='매출 (만원)',
    width=700,
    height=500
)

fig.show()

---

## 1.4 회귀계수 해석

### 비즈니스 관점의 해석

회귀계수는 단순한 숫자가 아니라 **의사결정에 직결되는 정보**입니다.

| 회귀계수 | 통계적 의미 | 비즈니스 의미 |
|---------|------------|---------------|
| 절편 (β₀) | X=0일 때 Y | 기본 매출 (광고 없을 때) |
| 기울기 (β₁) | X가 1 증가 시 Y 변화 | 광고비 1만원당 매출 증가 |

In [ ]:
# 회귀계수 활용: ROI 계산
slope = model.coef_[0]  # 기울기

# 광고 ROI = (매출 증가 / 광고비 투입) - 1
roi = (slope - 1)  # 1원 투입 대비 추가 수익

print("광고 투자 효율성 분석")
print("=" * 50)
print(f"광고비 1만원 투입 시 매출 증가: {slope:.2f}만원")
print(f"순수익 증가 (매출 - 광고비): {slope - 1:.2f}만원")
print(f"광고 ROI: {(slope - 1) / 1 * 100:.1f}%")

# 다양한 광고비 시나리오
print("\n광고비 시나리오 분석:")
scenarios = [100, 200, 300, 500, 1000]
for budget in scenarios:
    predicted = model.predict([[budget]])[0]
    profit = predicted - budget
    print(f"  광고비 {budget:,}만원 -> 예상매출 {predicted:,.0f}만원 (순이익: {profit:,.0f}만원)")

---

## 1.5 R-squared (결정계수)

### R-squared란?

**R² (결정계수)**: 모델이 종속변수의 변동을 얼마나 설명하는지 나타내는 지표

- **범위**: 0 ~ 1 (0% ~ 100%)
- **R² = 1**: 완벽한 예측 (모든 점이 회귀선 위)
- **R² = 0**: 모델이 아무것도 설명하지 못함

**수식**: R² = 1 - (SSE / SST) = 1 - (잔차제곱합 / 총변동)

### R² 해석 기준

| R² 값 | 해석 |
|-------|------|
| 0.9 이상 | 매우 높은 설명력 |
| 0.7 ~ 0.9 | 높은 설명력 |
| 0.4 ~ 0.7 | 보통 |
| 0.4 미만 | 낮은 설명력 |

In [ ]:
# R-squared 계산 (여러 방법)

# 방법 1: sklearn의 score() 메서드
r2_sklearn = model.score(X, y)

# 방법 2: r2_score 함수
r2_metric = r2_score(y, y_pred)

# 방법 3: 수동 계산
ss_total = np.sum((y - y.mean()) ** 2)  # 총 변동
ss_residual = np.sum((y - y_pred) ** 2)  # 잔차 변동
r2_manual = 1 - (ss_residual / ss_total)

print("R-squared (결정계수) 계산")
print("=" * 50)
print(f"sklearn score(): {r2_sklearn:.4f}")
print(f"r2_score(): {r2_metric:.4f}")
print(f"수동 계산: {r2_manual:.4f}")

print(f"\n해석: 광고비가 매출 변동의 {r2_sklearn*100:.1f}%를 설명합니다.")

### 💡 실무 예시: 모델 성능 리포트

In [ ]:
# 종합 모델 리포트
print("=" * 60)
print("회귀분석 모델 리포트")
print("=" * 60)

print("\n[모델 정보]")
print(f"  종속변수 (Y): 매출")
print(f"  독립변수 (X): 광고비")
print(f"  데이터 수: {len(y)}개")

print("\n[회귀 계수]")
print(f"  절편 (Intercept): {model.intercept_:.2f}")
print(f"  기울기 (광고비): {model.coef_[0]:.4f}")

print("\n[회귀식]")
print(f"  매출 = {model.intercept_:.2f} + {model.coef_[0]:.4f} x 광고비")

print("\n[모델 성능]")
print(f"  R-squared: {r2_sklearn:.4f} ({r2_sklearn*100:.1f}%)")

print("\n[비즈니스 인사이트]")
print(f"  - 광고비 1만원 증가 시 매출 약 {model.coef_[0]:.1f}만원 증가")
print(f"  - 광고비가 매출 변동의 {r2_sklearn*100:.1f}%를 설명")
print(f"  - 나머지 {(1-r2_sklearn)*100:.1f}%는 다른 요인에 의해 결정")

---

# Part 2: 심화 분석

---

## 2.1 다중 선형 회귀 (Multiple Linear Regression)

### 다중 회귀란?

**다중 선형 회귀**: 여러 개의 독립변수로 종속변수를 예측

**수식**: Y = β₀ + β₁X₁ + β₂X₂ + ... + βₙXₙ + ε

### 언제 사용하나?

- 단일 변수로 설명력이 부족할 때
- 여러 요인의 영향을 동시에 파악하고 싶을 때
- 변수들의 상대적 중요도를 비교하고 싶을 때

In [ ]:
# 부동산 가격 예측 데이터 (다중 회귀 예시)
np.random.seed(42)

n = 30
area = np.random.uniform(50, 150, n)  # 면적 (평)
floor = np.random.randint(1, 25, n)   # 층수
distance = np.random.uniform(0.1, 2.0, n)  # 역까지 거리 (km)

# 가격 생성 (면적, 층수 양의 영향 / 거리 음의 영향)
price = 5000 + 120 * area + 50 * floor - 800 * distance + np.random.normal(0, 500, n)

df_house = pd.DataFrame({
    '면적': area.round(1),
    '층수': floor,
    '역거리': distance.round(2),
    '가격': price.round(0)
})

print("부동산 데이터 (일부):")
print(df_house.head(10))
print(f"\n데이터 수: {len(df_house)}개")

In [ ]:
# 다중 회귀 모델 구축
X_multi = df_house[['면적', '층수', '역거리']]
y_price = df_house['가격']

# 모델 학습
model_multi = LinearRegression()
model_multi.fit(X_multi, y_price)

# 결과 출력
print("다중 선형 회귀 결과")
print("=" * 60)
print(f"절편: {model_multi.intercept_:.2f}")
print("\n회귀계수:")
for feature, coef in zip(X_multi.columns, model_multi.coef_):
    print(f"  {feature}: {coef:.4f}")

print(f"\nR-squared: {model_multi.score(X_multi, y_price):.4f}")

### 💡 회귀계수 해석 (다중 회귀)

**다중 회귀에서 회귀계수의 의미**: 다른 변수를 통제한 상태에서 해당 변수의 순수 효과

In [ ]:
# 회귀계수 비즈니스 해석
print("부동산 가격 결정 요인 분석")
print("=" * 60)

coef_names = ['면적', '층수', '역거리']
coef_values = model_multi.coef_

for name, coef in zip(coef_names, coef_values):
    if name == '면적':
        print(f"\n면적: {coef:.2f}만원/평")
        print(f"  -> 1평 증가 시 가격 약 {abs(coef):.0f}만원 상승")
    elif name == '층수':
        print(f"\n층수: {coef:.2f}만원/층")
        print(f"  -> 1층 높아질수록 가격 약 {abs(coef):.0f}만원 상승")
    elif name == '역거리':
        print(f"\n역거리: {coef:.2f}만원/km")
        print(f"  -> 역에서 1km 멀어질수록 가격 약 {abs(coef):.0f}만원 하락")

# 가격 예측 예시
print("\n" + "=" * 60)
print("가격 예측 예시")
print("=" * 60)

# 예시: 100평, 15층, 역에서 0.5km
sample = [[100, 15, 0.5]]
predicted_price = model_multi.predict(sample)[0]
print(f"\n조건: 면적 100평, 15층, 역거리 0.5km")
print(f"예상 가격: {predicted_price:,.0f}만원 (약 {predicted_price/10000:.1f}억원)")

---

## 2.2 statsmodels로 상세 분석

### statsmodels vs sklearn

| 라이브러리 | 강점 | 약점 | 주 사용처 |
|-----------|------|------|----------|
| sklearn | 예측 성능, 파이프라인 | 통계 검정 부족 | 머신러닝 |
| statsmodels | 상세 통계량, p-value | 예측 인터페이스 불편 | 통계 분석 |

### statsmodels.formula.api 사용법

R 스타일의 formula를 사용하여 회귀분석 수행

In [ ]:
# statsmodels로 회귀분석 (formula 방식)
# 컬럼명에 한글이 있으면 Q() 사용
model_stats = smf.ols('가격 ~ 면적 + 층수 + 역거리', data=df_house).fit()

# 요약 출력
print(model_stats.summary())

### summary() 해석 가이드

**상단 섹션**:
- **R-squared**: 모델 설명력
- **Adj. R-squared**: 변수 수 조정된 결정계수
- **F-statistic**: 모델 전체 유의성 검정
- **Prob (F-statistic)**: F-검정 p-value (0.05 미만이면 유의)

**계수 테이블**:
- **coef**: 회귀계수
- **std err**: 표준오차
- **t**: t-통계량
- **P>|t|**: p-value (0.05 미만이면 해당 변수 유의)
- **[0.025, 0.975]**: 95% 신뢰구간

In [ ]:
# 핵심 결과만 추출
print("회귀분석 핵심 결과")
print("=" * 60)

print("\n[모델 적합도]")
print(f"  R-squared: {model_stats.rsquared:.4f}")
print(f"  Adj. R-squared: {model_stats.rsquared_adj:.4f}")
print(f"  F-statistic: {model_stats.fvalue:.2f} (p-value: {model_stats.f_pvalue:.6f})")

print("\n[회귀계수와 유의성]")
coef_df = pd.DataFrame({
    '계수': model_stats.params,
    'p-value': model_stats.pvalues,
    '유의함': ['O' if p < 0.05 else 'X' for p in model_stats.pvalues]
})
print(coef_df.round(4))

print("\n[유의한 변수만]")
for name, pval in model_stats.pvalues.items():
    if pval < 0.05 and name != 'Intercept':
        coef = model_stats.params[name]
        print(f"  {name}: {coef:.4f} (p={pval:.4f})")

---

## 2.3 잔차 분석 (Residual Analysis)

### 회귀 가정 검증

선형 회귀는 다음 가정을 전제로 합니다:

1. **선형성**: X와 Y의 관계가 선형
2. **독립성**: 잔차들이 서로 독립
3. **등분산성**: 잔차의 분산이 일정
4. **정규성**: 잔차가 정규분포

### 잔차 플롯으로 가정 검증

In [ ]:
# 잔차 추출
residuals_multi = model_stats.resid
fitted_values = model_stats.fittedvalues

# 잔차 기본 통계
print("잔차 기본 통계")
print("=" * 50)
print(f"잔차 평균: {residuals_multi.mean():.4f} (0에 가까워야 함)")
print(f"잔차 표준편차: {residuals_multi.std():.2f}")
print(f"잔차 최소: {residuals_multi.min():.2f}")
print(f"잔차 최대: {residuals_multi.max():.2f}")

In [ ]:
# 잔차 플롯 4가지 (Plotly 사용)
fig = make_subplots(
    rows=2, cols=2,
    subplot_titles=(
        '1. 잔차 vs 예측값 (등분산성)',
        '2. 잔차 히스토그램 (정규성)',
        '3. Q-Q Plot (정규성)',
        '4. 잔차 순서도 (독립성)'
    )
)

# 1. 잔차 vs 예측값
fig.add_trace(
    go.Scatter(x=fitted_values, y=residuals_multi, mode='markers',
               marker=dict(color='royalblue', size=8)),
    row=1, col=1
)
fig.add_hline(y=0, line_dash='dash', line_color='red', row=1, col=1)

# 2. 잔차 히스토그램
fig.add_trace(
    go.Histogram(x=residuals_multi, nbinsx=10, marker_color='royalblue'),
    row=1, col=2
)

# 3. Q-Q Plot
sorted_residuals = np.sort(residuals_multi)
theoretical_quantiles = stats.norm.ppf(np.linspace(0.01, 0.99, len(sorted_residuals)))
fig.add_trace(
    go.Scatter(x=theoretical_quantiles, y=sorted_residuals, mode='markers',
               marker=dict(color='royalblue', size=8)),
    row=2, col=1
)
# 대각선 추가
fig.add_trace(
    go.Scatter(x=[-3, 3], y=[-3*residuals_multi.std(), 3*residuals_multi.std()],
               mode='lines', line=dict(color='red', dash='dash')),
    row=2, col=1
)

# 4. 잔차 순서도
fig.add_trace(
    go.Scatter(y=residuals_multi, mode='markers+lines',
               marker=dict(color='royalblue', size=8)),
    row=2, col=2
)
fig.add_hline(y=0, line_dash='dash', line_color='red', row=2, col=2)

fig.update_layout(height=700, width=900, title_text='잔차 분석 플롯', showlegend=False)
fig.show()

### 잔차 플롯 해석

1. **잔차 vs 예측값**: 패턴 없이 무작위로 퍼져 있어야 함 (등분산성)
2. **히스토그램**: 종 모양에 가까워야 함 (정규성)
3. **Q-Q Plot**: 점들이 대각선을 따라 배열되어야 함 (정규성)
4. **순서도**: 특정 패턴 없이 무작위해야 함 (독립성)

In [ ]:
# 정규성 검정 (Shapiro-Wilk)
shapiro_stat, shapiro_p = stats.shapiro(residuals_multi)

print("정규성 검정 (Shapiro-Wilk)")
print("=" * 50)
print(f"통계량: {shapiro_stat:.4f}")
print(f"p-value: {shapiro_p:.4f}")

if shapiro_p > 0.05:
    print("결론: 잔차가 정규분포를 따른다고 볼 수 있습니다.")
else:
    print("주의: 잔차가 정규분포를 따르지 않을 수 있습니다.")

---

## 2.4 회귀 평가 지표 (RMSE, MAE)
⭐나만의 언어로 언제 어떤 회귀 평가지표를 쓰는지 잘 정리하기⭐
⭐내가 코드를 직접 작성은 못할지라도 쓰인 코드를 보며 해석은 가능한지 체크하기⭐

### 평가 지표 종류

| 지표 | 수식 | 특징 |
|------|------|------|
| **MAE** | Σ|y-ŷ| / n | 직관적, 이상치에 덜 민감 |
| **MSE** | Σ(y-ŷ)² / n | 큰 오차에 페널티 |
| **RMSE** | √MSE | 원래 단위로 해석 가능 |
| **R²** | 1 - SSE/SST | 설명력 (0~1) |

In [ ]:
# 예측값 생성
y_pred_multi = model_multi.predict(X_multi)

# 평가 지표 계산
mae = mean_absolute_error(y_price, y_pred_multi)
mse = mean_squared_error(y_price, y_pred_multi)
rmse = np.sqrt(mse)
r2 = r2_score(y_price, y_pred_multi)

print("회귀 모델 평가 지표")
print("=" * 50)
print(f"MAE (평균 절대 오차): {mae:.2f}만원")
print(f"MSE (평균 제곱 오차): {mse:.2f}")
print(f"RMSE (평균 제곱근 오차): {rmse:.2f}만원")
print(f"R² (결정계수): {r2:.4f}")

print("\n해석:")
print(f"  - 평균적으로 예측 오차는 약 {mae:.0f}만원")
print(f"  - 오차의 표준편차는 약 {rmse:.0f}만원")
print(f"  - 모델이 가격 변동의 {r2*100:.1f}%를 설명")

### 💡 실무 예시: 모델 비교

In [ ]:
# 단순 회귀 vs 다중 회귀 비교
print("모델 비교: 단순 회귀 vs 다중 회귀")
print("=" * 60)

# 단순 회귀 (면적만)
model_simple = LinearRegression()
model_simple.fit(df_house[['면적']], y_price)
y_pred_simple = model_simple.predict(df_house[['면적']])

# 단순 회귀 평가
mae_simple = mean_absolute_error(y_price, y_pred_simple)
rmse_simple = np.sqrt(mean_squared_error(y_price, y_pred_simple))
r2_simple = r2_score(y_price, y_pred_simple)

# 비교 테이블
comparison = pd.DataFrame({
    '모델': ['단순 회귀 (면적)', '다중 회귀 (면적+층수+역거리)'],
    'MAE (만원)': [mae_simple, mae],
    'RMSE (만원)': [rmse_simple, rmse],
    'R²': [r2_simple, r2]
})

print(comparison.round(2).to_string(index=False))

print(f"\n결론:")
print(f"  - R² 개선: {r2_simple:.4f} -> {r2:.4f} (+{(r2-r2_simple)*100:.1f}%p)")
print(f"  - MAE 감소: {mae_simple:.0f} -> {mae:.0f}만원 ({(mae_simple-mae)/mae_simple*100:.1f}% 개선)")
print(f"  - 다중 회귀가 더 정확한 예측 제공")

### 💡 실무 예시: 실제값 vs 예측값 시각화

In [ ]:
# 실제값 vs 예측값 산점도
fig = go.Figure()

# 실제 vs 예측
fig.add_trace(go.Scatter(
    x=y_price,
    y=y_pred_multi,
    mode='markers',
    name='예측값',
    marker=dict(size=10, color='royalblue')
))

# 완벽한 예측선 (대각선)
min_val = min(y_price.min(), y_pred_multi.min())
max_val = max(y_price.max(), y_pred_multi.max())
fig.add_trace(go.Scatter(
    x=[min_val, max_val],
    y=[min_val, max_val],
    mode='lines',
    name='완벽한 예측',
    line=dict(color='red', width=2, dash='dash')
))

fig.update_layout(
    title=f'실제값 vs 예측값 (R² = {r2:.4f})',
    xaxis_title='실제 가격 (만원)',
    yaxis_title='예측 가격 (만원)',
    width=600,
    height=500
)

fig.show()

---

## 🎯 실습 퀴즈

**난이도**: ⭐ (쉬움) ~ ⭐⭐⭐⭐⭐ (어려움)

---

### Q1. 단순 선형 회귀 모델 구축 ⭐

**문제**: 공부시간과 시험점수 데이터로 단순 선형 회귀 모델을 만들고, 회귀계수를 출력하세요.

```python
study_hours = [2, 3, 4, 5, 6, 7, 8, 9, 10]
test_scores = [55, 60, 65, 70, 75, 80, 85, 88, 92]
```

**출력**: 절편, 기울기, 회귀식

In [ ]:
study_hours = [2, 3, 4, 5, 6, 7, 8, 9, 10]
test_scores = [55, 60, 65, 70, 75, 80, 85, 88, 92]
# 여기에 코드를 작성하세요


### Q2. 예측값 계산 ⭐⭐

**문제**: Q1에서 만든 모델로 공부시간 12시간일 때 예상 점수를 예측하세요.

**출력**: 예측 점수

In [ ]:
# Q1의 모델을 사용하세요
# 여기에 코드를 작성하세요


### Q3. R-squared 계산 ⭐⭐

**문제**: Q1 모델의 R-squared를 계산하고 해석하세요.

**출력**: R² 값, 해석 문장

In [ ]:
# 여기에 코드를 작성하세요


### Q4. px.scatter로 회귀선 시각화 ⭐⭐⭐

**문제**: Q1 데이터를 `px.scatter`와 `trendline='ols'`로 시각화하세요.

**요구사항**:
- 제목: "공부시간 vs 시험점수"
- x축: "공부시간 (시간)"
- y축: "시험점수 (점)"

In [ ]:
# 여기에 코드를 작성하세요


### Q5. 잔차 계산 ⭐⭐⭐

**문제**: Q1 모델의 잔차를 계산하고, 잔차의 합과 평균을 출력하세요.

**출력**: 각 데이터의 잔차, 잔차 합계, 잔차 평균

In [ ]:
# 여기에 코드를 작성하세요


---

## Part 2 심화 퀴즈

---

### Q6. 다중 회귀 모델 구축 ⭐⭐⭐

**문제**: 다음 매장 데이터로 다중 회귀 모델을 만들고 각 변수의 회귀계수를 해석하세요.

```python
store_data = pd.DataFrame({
    '면적': [50, 80, 120, 60, 100, 90, 70, 110, 85, 95],  # 평수
    '직원수': [3, 5, 8, 4, 7, 6, 4, 7, 5, 6],
    '월매출': [2000, 3200, 5000, 2500, 4200, 3800, 2800, 4500, 3400, 3900]  # 만원
})
```

**출력**: 각 변수의 회귀계수, 비즈니스 해석

In [ ]:
store_data = pd.DataFrame({
    '면적': [50, 80, 120, 60, 100, 90, 70, 110, 85, 95],
    '직원수': [3, 5, 8, 4, 7, 6, 4, 7, 5, 6],
    '월매출': [2000, 3200, 5000, 2500, 4200, 3800, 2800, 4500, 3400, 3900]
})
# 여기에 코드를 작성하세요


### Q7. statsmodels 요약 분석 ⭐⭐⭐⭐

**문제**: Q6 데이터로 statsmodels를 사용해 회귀분석을 수행하고, p-value로 유의한 변수를 판단하세요.

**출력**: summary() 결과, 유의한 변수 목록

In [ ]:
# 여기에 코드를 작성하세요


### Q8. 평가 지표 계산 ⭐⭐⭐⭐

**문제**: Q6 모델의 MAE, RMSE, R²를 계산하고 해석하세요.

**출력**: 세 가지 평가 지표와 각각의 해석

In [ ]:
# 여기에 코드를 작성하세요


### Q9. 잔차 플롯 시각화 ⭐⭐⭐⭐

**문제**: Q6 모델의 잔차 vs 예측값 플롯을 Plotly로 그리고, 등분산성 가정을 평가하세요.

**요구사항**:
- y=0 기준선 표시
- 등분산성 가정 충족 여부 판단

In [ ]:
# 여기에 코드를 작성하세요


### Q10. 종합 분석 프로젝트 ⭐⭐⭐⭐⭐

**문제**: 온라인 쇼핑몰 매출 예측 모델을 구축하고 완전한 분석 리포트를 작성하세요.

```python
shop_data = pd.DataFrame({
    '광고비': [100, 150, 200, 120, 180, 220, 160, 190, 140, 210],  # 만원
    '방문자수': [1200, 1800, 2500, 1400, 2200, 2800, 1900, 2400, 1600, 2600],
    '할인율': [5, 10, 15, 7, 12, 18, 10, 14, 8, 16],  # %
    '매출': [800, 1200, 1700, 950, 1500, 1900, 1300, 1650, 1100, 1800]  # 만원
})
```

**요구사항**:
1. 다중 회귀 모델 구축
2. 회귀계수 해석 (비즈니스 언어)
3. 모델 평가 (R², MAE, RMSE)
4. 실제값 vs 예측값 시각화
5. 비즈니스 인사이트 도출

In [ ]:
shop_data = pd.DataFrame({
    '광고비': [100, 150, 200, 120, 180, 220, 160, 190, 140, 210],
    '방문자수': [1200, 1800, 2500, 1400, 2200, 2800, 1900, 2400, 1600, 2600],
    '할인율': [5, 10, 15, 7, 12, 18, 10, 14, 8, 16],
    '매출': [800, 1200, 1700, 950, 1500, 1900, 1300, 1650, 1100, 1800]
})

# 여기에 코드를 작성하세요


---

## 📊 학습 정리

### Part 1: 기초 핵심 요약

| 개념 | 정의 | 핵심 포인트 |
|------|------|-------------|
| 회귀분석 | X로 Y를 예측하는 모델링 | 관계를 수식으로 표현 |
| 단순 선형 회귀 | Y = β₀ + β₁X | 하나의 독립변수 |
| 최소제곱법 (OLS) | 잔차 제곱합 최소화 | 회귀계수 추정 방법 |
| 절편 (β₀) | X=0일 때 Y | 기본값 |
| 기울기 (β₁) | X가 1 증가 시 Y 변화 | 영향력 크기 |
| R-squared | 모델 설명력 | 0~1 (높을수록 좋음) |

### Part 2: 심화 핵심 요약

| 기법 | 용도 | Python 함수 | 언제 쓰나? |
|------|------|-------------|------------|
| 다중 회귀 | 여러 변수로 예측 | `LinearRegression()` | 복합 요인 분석 |
| statsmodels | 상세 통계 분석 | `smf.ols().fit()` | p-value, 신뢰구간 필요 |
| 잔차 분석 | 가정 검증 | `.resid` | 모델 진단 |
| MAE | 평균 절대 오차 | `mean_absolute_error()` | 직관적 해석 |
| RMSE | 평균 제곱근 오차 | `np.sqrt(mean_squared_error())` | 원래 단위로 해석 |

### 💡 실무 팁

1. **회귀계수 해석**: 기울기는 "X가 1 증가하면 Y가 β만큼 변화"로 해석
2. **R² 주의**: 변수 추가하면 무조건 증가 → Adjusted R² 함께 확인
3. **다중공선성**: 독립변수들끼리 상관이 높으면 회귀계수 불안정
4. **잔차 검토 필수**: 패턴 있으면 모델 가정 위반 → 모델 수정 필요
5. **외삽 주의**: 학습 데이터 범위 밖 예측은 신뢰하기 어려움
6. **비즈니스 언어**: 통계 결과를 비즈니스 인사이트로 번역
7. **시각화 필수**: 숫자만으로 판단하지 말고 항상 그래프 확인

### 📈 회귀분석 워크플로우

```
1. 데이터 탐색 (EDA)
   ↓
2. 모델 구축 (단순/다중 회귀)
   ↓
3. 회귀계수 확인 및 해석
   ↓
4. 모델 평가 (R², MAE, RMSE)
   ↓
5. 잔차 분석 (가정 검증)
   ↓
6. 예측 및 비즈니스 활용
```